# Exercicio pra calcular imoveis 

In [40]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [41]:
requests.get('https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145').content

b'\n<!DOCTYPE html>\n<html lang="pt-BR">\n<head><link rel="preconnect" href="https://www.google-analytics.com"/><link rel="preconnect" href="https://www.google.com"/><link rel="preconnect" href="https://data.mercadolibre.com"/><link rel="preconnect" href="https://http2.mlstatic.com"/><link rel="preconnect" href="https://assets.mlstatic.com"/><link rel="preconnect" href="https://stats.g.doubleclick.net"/><link rel="preconnect" href="https://analytics.mercadolivre.com.br"/><link rel="preconnect" href="https://analytics.mercadolivre.com"/><link rel="preconnect" href="https://www.google.com.br"/><script type=\'text/javascript\' >window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"3009922991","applicationID":"523456907","applicationTime":1191.259702,"transactionName":"bgRaYENYWBdWABdfXVdOfUxBS1MXRAkQGXV8NRcbGw==","queueTime":0,"ttGuid":"b7548d017489f081","agentToken":null}; (window.NREUM||(NREUM={})).loader_con

In [42]:
ZONAS = ['norte', 'sul', 'leste', 'oeste']
PAGINAS = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
URL_ML = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

# Podemos definir dois padrões regex, um para o numero de quartos e outro para a area
re_quarto = '\| (.*) quarto'
re_area = '> (.*) m²'
re_quarto_area_ausente = "> (.*) quarto"

In [43]:
def coletar_dados_ml(url, zona):
  c = requests.get(url).content
  soup = BeautifulSoup(c)
  atributos = soup.find_all('div', {"class": "item__attrs"})
  precos = soup.find_all('span', {"class": "price__fraction"})

  lista_precos = []
  for s in precos:
    p = re.findall('<span class="price__fraction">(.*)</span>', str(s))
    lista_precos.append(int(p[0].replace('.','')))

  # Inicializar duas listas vazias, uma para a area e outra para o numero de quartos
  lista_areas = []
  lista_quartos = []
  lista_zonas = []
  for a in atributos:
    # Extrair area e número de quartos
    n_quartos = re.findall(re_quarto, str(a))
    area = re.findall(re_area, str(a))
    
    # Atribuir np.nan quando o número de quartos estiver ausente 
    if n_quartos==[]:
      n_quartos = re.findall(re_quarto_area_ausente, str(a))

      if n_quartos==[]:
        n_quartos=np.nan
      else:
        n_quartos=n_quartos[0]
  
    else:
        n_quartos=n_quartos[0]

    if area==[]:
      area=np.nan
    else:
      area=area[0]
    #print(n_quartos)

    # Anexar às listas acima
    lista_quartos.append(n_quartos)
    lista_areas.append(area)
    lista_zonas.append(zona)

  dados = {
      'zona': lista_zonas,
      'quartos': lista_quartos,
      'area': lista_areas,
      'preco': lista_precos
      }

  df = pd.DataFrame(dados)
  return df

In [44]:
lista_dfs = []
for zona in ZONAS:
  for pg in PAGINAS:
    url = URL_ML.format(zona, pg)
    print('Carregando a URL', url)

    df = coletar_dados_ml(url, zona)
    lista_dfs.append(df)
    sleep(1)
    

Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com

In [32]:
dados = pd.concat(lista_dfs)

In [ ]:
dados

,zona,quartos,area,preco
0,norte,1,70,600
1,norte,2,70,850
2,norte,1,40,850
3,norte,1,50,950
4,norte,1,160,790
...,...,...,...,...
43,oeste,3,200,15000
44,oeste,2,120,6000
45,oeste,4,350,7900
46,oeste,2,240,4600


In [45]:
dados['quartos'] = dados['quartos'].astype(float)
dados['area'] = dados['area'].astype(float)

In [47]:
dados.to_csv('dados_calculadora_imoveis_aula1\.csv', index=False)

In [48]:
pd.read_csv("dados_calculadora_imoveis_aula.csv")

,zona,quartos,area,preco
0,norte,1.0,70.0,600
1,norte,2.0,70.0,850
2,norte,1.0,40.0,850
3,norte,1.0,50.0,950
4,norte,1.0,160.0,790
...,...,...,...,...
1147,oeste,3.0,200.0,15000
1148,oeste,2.0,120.0,6000
1149,oeste,4.0,350.0,7900
1150,oeste,2.0,240.0,4600


In [39]:
ls

Atividade-Calculadora-Imoveis.ipynb
Atividade_Coleta_e_Manipulação_ML.ipynb
README.md
dados_calculadora_imoveis_Desafio.csv
dados_calculadora_imoveis_aula.csv
dados_calculadora_imoveis_aula1.csv
